In [1]:
import pandas as pd
import numpy as np
import re


In [2]:
train= pd.read_csv("train.csv")

C:\Users\moi\AppData\Local\Temp\ipykernel_18548\3625669964.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  train= pd.read_csv("train.csv")


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 38 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  int64  
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             97217 non-null   float64
 8   Monthly_Inhand_Salary     100000 non-null  float64
 9   Num_Bank_Accounts         100000 non-null  object 
 10  Num_Credit_Card           100000 non-null  object 
 11  Interest_Rate             100000 non-null  float64
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [3]:
def replace_outliers_with_na(train, column_name):
    # Calculer le premier quartile (Q1) et le troisième quartile (Q3)
    Q1 = train[column_name].quantile(0.25)
    Q3 = train[column_name].quantile(0.75)

    # Calculer l'intervalle interquartile (IQR)
    IQR = Q3 - Q1

    # Définir les limites pour détecter les outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Remplacer les outliers par une valeur manquante
    train.loc[(train[column_name] < lower_bound) | (train[column_name] > upper_bound), column_name] = np.nan

# Fonction mise à jour pour gérer les valeurs NaN
def extract_and_count_loan_types(df, column_name):
    # Extraire tous les types de crédits uniques
    all_loan_types = set()
    for loans in df[column_name].dropna():  # Ignorer les NaN
        for loan_type in loans.split(', '):
            all_loan_types.add(loan_type.strip())

    # Créer de nouvelles colonnes pour chaque type de crédit et initialiser à 0
    for loan_type in all_loan_types:
        df[loan_type] = 0

    # Compter les occurrences de chaque type de crédit pour chaque ligne
    for index, row in df.iterrows():
        if pd.notna(row[column_name]):  # Vérifier que la valeur n'est pas NaN
            loan_types = row[column_name].split(', ')
            for loan_type in loan_types:
                df.at[index, loan_type.strip()] += 1

    return df




def convert_to_years_with_rounding(duration_str):
    
    if not isinstance(duration_str, str):
        return None  

    
    pattern = re.compile(r'(?:(\d+)\s*Years?)?\s*and\s*(?:(\d+)\s*Months?)?')
    match = pattern.search(duration_str)
    if not match:
        return None

    
    years = int(match.group(1)) if match.group(1) else 0
    months = int(match.group(2)) if match.group(2) else 0

    
    total_years = round((years * 12))

In [4]:
def preprocessing(data):
  # Age
  data['Age'] = data['Age'].str.replace('_', '')
  data['Age']=pd.to_numeric(data['Age'])
  data[(data['Age'] > 90) | (data['Age'] < 0)]
  data['Age'] = data['Age'].fillna(data['Age'].median())

  # Occupation
  data.loc[(data['Occupation'] =="___"), 'Occupation'] ="manquant"

  # Annual_Income
  data['Annual_Income'] = data['Annual_Income'].str.replace('_', '')
  data['Annual_Income']=pd.to_numeric(data['Annual_Income'])
  replace_outliers_with_na(data, 'Annual_Income')

  # Monthly_Inhand_Salary
  median_salaries_by_occupation = data.groupby('Occupation')['Monthly_Inhand_Salary'].median()
  for occupation, median_salary in median_salaries_by_occupation.items():
      data.loc[data['Occupation'] == occupation, 'Monthly_Inhand_Salary'] = data.loc[data['Occupation'] == occupation, 'Monthly_Inhand_Salary'].fillna(median_salary)

  # Num_Bank_Accounts
  data.loc[data['Num_Bank_Accounts'] < 1, 'Num_Bank_Accounts'] = np.nan
  replace_outliers_with_na(data, 'Num_Bank_Accounts')
  data['Num_Bank_Accounts'].fillna('manquant',inplace=True)
  data['Num_Bank_Accounts'] = data['Num_Bank_Accounts'].astype(str).str.replace(r'\.0\b', '')

  # Num_Credit_Card

  replace_outliers_with_na(data, 'Num_Credit_Card')
  data['Num_Credit_Card'] = data['Num_Credit_Card'].astype(str).str.replace(r'\.0\b', '')

  # Interest_Rate
  replace_outliers_with_na(data, 'Interest_Rate')
  data['Interest_Rate'] = data['Interest_Rate'].fillna(data['Interest_Rate'].median())

  # Num_of_Loan
  data['Num_of_Loan'] = data['Num_of_Loan'].str.replace('_', '')
  data['Num_of_Loan']=pd.to_numeric(data['Num_of_Loan'])
  replace_outliers_with_na(data, 'Num_of_Loan')
  data['Num_of_Loan'].fillna('manquant',inplace=True)
  data['Num_of_Loan'] = data['Num_of_Loan'].astype(str).str.replace(r'\.0\b', '')

  # Type_of_Loan
  data['Type_of_Loan'] = data['Type_of_Loan'].str.replace('and', '')
  data = extract_and_count_loan_types(data, 'Type_of_Loan')

  # Delay_from_due_date
  data.loc[data['Delay_from_due_date'] < 0, 'Delay_from_due_date'] = np.nan
  data['Delay_from_due_date'] = data['Delay_from_due_date'].fillna(data['Delay_from_due_date'].median())

  #Num_of_Delayed_Payment
  data['Num_of_Delayed_Payment'] = data['Num_of_Delayed_Payment'].str.replace('_', '').astype(float)

  #Changed_Credit_Limit
  data['Changed_Credit_Limit'] = data['Changed_Credit_Limit'].replace('_', np.nan, regex=True).astype(float)

  #Credit_Mix
  data.loc[(data['Credit_Mix'] =="_"), 'Credit_Mix'] ="manquant"

  #Outstanding_Debt
  data['Outstanding_Debt'] = data['Outstanding_Debt'].str.replace('_', '').astype(float)

  # Credit_Utilization_Ratio

  # Credit history year
  data['credit_history_year'] = data['Credit_History_Age'].apply(convert_to_years_with_rounding)
  data['credit_history_year'] = data['credit_history_year'].fillna(data['credit_history_year'].median())

  # Total EMI_per_month
  replace_outliers_with_na(data, 'Total_EMI_per_month')
  data['Total_EMI_per_month'] = data['Total_EMI_per_month'].fillna(data['Total_EMI_per_month'].median())

  # Amount invested monthly
  data['Amount_invested_monthly'] = data['Amount_invested_monthly'].replace('__10000__', np.nan)
  data['Amount_invested_monthly']=pd.to_numeric(data['Amount_invested_monthly'])

  # Monthly_Balance
  data['Monthly_Balance']=data['Monthly_Balance'].replace('__-333333333333333333333333333__',np.nan)
  data['Monthly_Balance']=pd.to_numeric(data['Monthly_Balance'])
  data['Monthly_Balance'] = data['Monthly_Balance'].fillna(data['Monthly_Balance'].median())

  return data

data_process=preprocessing(train)

C:\Users\moi\AppData\Local\Temp\ipykernel_18548\2202922075.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'manquant' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data['Num_Bank_Accounts'].fillna('manquant',inplace=True)
C:\Users\moi\AppData\Local\Temp\ipykernel_18548\2202922075.py:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'manquant' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data['Num_of_Loan'].fillna('manquant',inplace=True)
D:\ANACONDA\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [5]:
data_process

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Home Equity Loan,Personal Loan,Debt Consolidation Loan,Not Specified,Mortgage Loan,Student Loan,Payday Loan,Auto Loan,Credit-Builder Loan,credit_history_year
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3.0,...,1,1,0,0,0,0,0,1,1,NaN
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,3260.465000,3.0,...,1,1,0,0,0,0,0,1,1,NaN
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,3260.465000,3.0,...,1,1,0,0,0,0,0,1,1,NaN
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,3260.465000,3.0,...,1,1,0,0,0,0,0,1,1,NaN
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3.0,...,1,1,0,0,0,0,0,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0x25fe9,CUS_0x942c,April,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4.0,...,0,0,0,0,0,1,0,1,0,NaN
99996,0x25fea,CUS_0x942c,May,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4.0,...,0,0,0,0,0,1,0,1,0,NaN
99997,0x25feb,CUS_0x942c,June,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4.0,...,0,0,0,0,0,1,0,1,0,NaN
99998,0x25fec,CUS_0x942c,July,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4.0,...,0,0,0,0,0,1,0,1,0,NaN


In [7]:

X=data_process.drop(['credit_history_year','Credit_Score','Type_of_Loan'],axis=1)

y=data_process['Credit_Score']

cat_features = list(X.select_dtypes(include=['object']).columns)



In [8]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Remplir les valeurs NaN pour les variables catégorielles dans les ensembles d'entraînement et de test
for cat_feature in cat_features:
    X_train[cat_feature].fillna('missing', inplace=True)
    X_test[cat_feature].fillna('missing', inplace=True)

# Création et entraînement du modèle CatBoost avec les variables catégorielles spécifiées
model = CatBoostClassifier(
    iterations=1500,
    learning_rate=0.1,
    depth=6,
    cat_features=cat_features,
    eval_metric='Accuracy',
    verbose=100
)

model.fit(X_train, y_train, eval_set=(X_test, y_test), use_best_model=True)

# Prédiction sur l'ensemble de test et évaluation de la précision
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy:.4f}')


0:	learn: 0.7707375	test: 0.8092000	best: 0.8092000 (0)	total: 562ms	remaining: 14m 2s
100:	learn: 0.8024625	test: 0.8390500	best: 0.8392000 (99)	total: 47.3s	remaining: 10m 55s
200:	learn: 0.8090125	test: 0.8422000	best: 0.8423000 (199)	total: 1m 32s	remaining: 9m 55s
300:	learn: 0.8132250	test: 0.8437000	best: 0.8437000 (299)	total: 2m 15s	remaining: 8m 59s
400:	learn: 0.8160000	test: 0.8445000	best: 0.8447000 (397)	total: 3m 19s	remaining: 9m 7s
500:	learn: 0.8183875	test: 0.8453000	best: 0.8454500 (488)	total: 4m 22s	remaining: 8m 43s
600:	learn: 0.8206750	test: 0.8453000	best: 0.8456500 (520)	total: 5m 11s	remaining: 7m 46s
700:	learn: 0.8233375	test: 0.8453000	best: 0.8457000 (648)	total: 6m 7s	remaining: 6m 58s
800:	learn: 0.8251375	test: 0.8454500	best: 0.8463000 (747)	total: 7m 15s	remaining: 6m 19s
900:	learn: 0.8279000	test: 0.8455500	best: 0.8463000 (747)	total: 8m 16s	remaining: 5m 30s
1000:	learn: 0.8299125	test: 0.8459500	best: 0.8463000 (747)	total: 9m 8s	remaining: 4m 